# 필요한 라이브러리 import

In [20]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf
import joblib
warnings.filterwarnings(action='ignore')

# 모델,스케일러,데이터 객체 로드

In [21]:
with open('./models/forward/forward_qt.pickle','rb') as f:
    forward_po = pickle.load(f)

forward_model = load_model('./models/forward/forward_nn_high.h5')

premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'파울 획득':np.int64,'태클 성공률':np.int64})

# 데이터 변환

In [22]:
forward_df = premier_df[premier_df['포지션']=='공격수']
forward_df = forward_df.reset_index(drop=True)

# Goal_Shoot_linear
forward_df['GSL'] = (forward_df['경기당 유효 슈팅/90']+0.5) + (forward_df['경기당 슈팅/90']) + (forward_df['골/90']*1.3)

# PSTL = Pass sucess try linear
forward_df['PSTL'] = (forward_df['경기당 패스 성공/90']) + (forward_df['경기당 패스 시도/90'])

# CACCL = Cross Attemp Cross Complete Linear
forward_df['CACCL'] = (forward_df['크로스 시도/90']) + (forward_df['크로스 성공률'])


# 독립 변수, 종속 변수 상세
independent_value = {'CACCL','GSL','기회 창출/90','도움/90','드리블 성공/90','태클 성공/90','파울 획득','헤더 성공/90'}

dependent_value = {'평균획득승점/경기'}

# 독립변수 스케일링

In [23]:
forward_df.loc[:,independent_value] = forward_po.transform(forward_df.loc[:,independent_value])

# PPG 예측

In [24]:
forward_df['예측PPG']=forward_model.predict(forward_df.loc[:,independent_value])

# WAR 계산 (PPG - Min_PPG) + 1

In [25]:
forward_min = np.min(forward_df['예측PPG'].tolist())
forward_df['WAR'] = forward_df['예측PPG'].map(lambda x : x-forward_min+1)

# 독립변수를 알아볼 수 있게 다시 복원

In [26]:
forward_df.loc[:,independent_value] = forward_po.inverse_transform(forward_df.loc[:,independent_value])

# WAR 확인

In [27]:
forward_war = forward_df.sort_values(by='WAR',ascending=False).copy()
forward_war = forward_war.reset_index(drop=True)

In [28]:
forward_war[['이름','시즌','골/90', '도움/90','WAR']].sort_values(by='WAR',ascending=False).head(10)

,이름,시즌,골/90,도움/90,WAR
0,Romelu Lukaku,2021-2022,0.655738,0.000000,12.971981
1,Diogo Jota,2021-2022,0.372561,0.159669,9.310770
2,Alexandre Lacazette,2021-2022,0.701950,0.000000,8.898141
3,Dominic Calvert-Lewin,2021-2022,0.527344,0.050223,8.161325
4,Harry Kane,2021-2022,0.539439,0.258931,8.100398
5,Kelechi Ịheanachọ,2021-2022,0.459966,0.000000,8.015787
6,Edinson Cavani,2021-2022,0.836098,0.119443,8.010510
7,Chris Wood,2021-2022,0.558691,0.025395,7.567613
8,Che Adams,2021-2022,0.555556,0.029240,7.502624
9,Cristiano Ronaldo,2021-2022,0.561831,0.240785,7.256310
